In [2]:
import pandas as pd
import string
import re
from pyvi import ViTokenizer

In [3]:
df_train = pd.read_csv('G:\\VietNamese-Sentiment-Analyst\\data\\data_original\\Train.csv')
df_test = pd.read_csv('G:\\VietNamese-Sentiment-Analyst\\data\\data_original\\Test.csv')
df_dev = pd.read_csv('G:\\VietNamese-Sentiment-Analyst\\data\\data_original\\Dev.csv')

In [4]:
df_train.columns

Index(['index', 'comment', 'n_star', 'date_time', 'label'], dtype='object')

In [5]:
df_train.head(5)

,index,comment,n_star,date_time,label
0,0,Mới mua máy này Tại thegioididong thốt nốt cảm...,5,2 tuần trước,{CAMERA#Positive};{FEATURES#Positive};{BATTERY...
1,1,Pin kém còn lại miễn chê mua 8/3/2019 tình trạ...,5,14/09/2019,{BATTERY#Negative};{GENERAL#Positive};{OTHERS};
2,2,Sao lúc gọi điện thoại màn hình bị chấm nhỏ nh...,3,17/08/2020,{FEATURES#Negative};
3,3,"Mọi người cập nhật phần mềm lại , nó sẽ bớt tố...",3,29/02/2020,{FEATURES#Negative};{BATTERY#Neutral};{GENERAL...
4,4,"Mới mua Sài được 1 tháng thấy pin rất trâu, Sà...",5,4/6/2020,{BATTERY#Positive};{PERFORMANCE#Positive};{SER...


In [6]:
df_train.shape

(7786, 5)

In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7786 entries, 0 to 7785
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      7786 non-null   int64 
 1   comment    7786 non-null   object
 2   n_star     7786 non-null   int64 
 3   date_time  7786 non-null   object
 4   label      7786 non-null   object
dtypes: int64(2), object(3)
memory usage: 304.3+ KB


In [8]:
df_train.describe()

,index,n_star
count,7786.000000,7786.000000
mean,3892.500000,3.706011
std,2247.768931,1.508531
min,0.000000,1.000000
25%,1946.250000,3.000000
50%,3892.500000,4.000000
75%,5838.750000,5.000000
max,7785.000000,5.000000


In [9]:
print(df_train['comment'].values[95:98])

['Điện thoại khá tốt pin trâu khá mượt bắt wifi cực tốt chỉ là thỉnh thoảng máy bị đơ phải thoát ra vào lại và máy ko cập nhật lên miul 12'
 'Lúc trước nghe bảo điện thoại chơi game nóng, nhưng mua về chiến liên quân 3 4h liên tục thì chỉ thấy ấm không nóng như lời đồn, những thứ khác thì vũng chẳng có gì để nói vì quá ngon rồi'
 '1. Máy nếu không chơi game, lướt web..., ít sử dụng thì giữ Pin cao nhất được khoảng 1 ngày 16 tiếng ( Tính luôn cả lúc ngủ nhé vì khi ngủ thì chẳng động gì tới ) . Không được 2 ngày như mn nói đâu \n2. Nhạc nghe cứ cà giật cà giật rất khó chịu luôn\n3. Quay video bị mờ chán \nCòn lại okie 🙆']


In [10]:
def process_numbers(sentence):
    # Chuẩn hóa giá
    sentence = re.sub(r'(\d+)\s*(triệu|tr|trieu)', r'\1 triệu', sentence, flags=re.IGNORECASE)
    
    # Chuẩn hóa số điện thoại
    sentence = re.sub(r'\b\d{10,11}\b', '[PHONE_NUMBER]', sentence)
    
    # Giữ nguyên các số khác
    return sentence

In [11]:
import unicodedata

def remove_punctuation(sentence):
    # Tokenization
    sentence = ViTokenizer.tokenize(sentence)   
    # Chuẩn hóa dấu tiếng Việt
    sentence = unicodedata.normalize('NFC', sentence)
    # Xử lý URL
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    sentence = url_pattern.sub('[URL]', sentence)
    # Xử lý số
    sentence = process_numbers(sentence)
    # Loại bỏ dấu câu và ký tự đặc biệt
    sentence = re.sub(r"[-()\"#/@;:<>{}`+=~|*'.!?,.]", "", sentence)
    # Xử lý ký tự đặc biệt và dấu tiếng Việt
    sentence = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ]',' ',sentence)
    # Chuyển về chữ thường
    sentence = sentence.lower()
    # Chuẩn hóa khoảng trắng
    sentence = re.sub(r'\s+', ' ', sentence).strip()
    # Loại bỏ emoji
    emoji_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
            u"\U00002500-\U00002BEF"  # chinese char
            u"\U00002702-\U000027B0"
            u"\U000024C2-\U0001F251"
            u"\U0001f926-\U0001f937"
            u"\U00010000-\U0010ffff"
            u"\u2640-\u2642"
            u"\u2600-\u2B55"
            u"\u200d"
            u"\u23cf"
            u"\u23e9"
            u"\u231a"
            u"\ufe0f"  # dingbats
            u"\u3030"
                            "]+", flags=re.UNICODE)
    sentence = re.sub(emoji_pattern, '', sentence)
    return sentence

In [12]:
def read_filestopwords():
    with open('G:\\VietNamese-Sentiment-Analyst\\data\\data_stopword\\vietnamese-stopwords.txt', 'r', encoding='utf-8') as file:
        lines = file.readlines()
        words = [line.split('\n')[0] for line in lines]
    return words

In [13]:
def remove_stopword(comment):
  stop_words = read_filestopwords()
  filtered = [word for word in comment.split() if word not in stop_words]
  return ' '.join(filtered)

In [14]:
def remove_repeated_words(text):
    words = text.split()
    new_words = []
    for i in range(len(words)):
        if i == 0 or words[i] != words[i-1]:
            new_words.append(words[i])
    return ' '.join(new_words)

In [15]:
df_train['comment'] = df_train['comment'].apply(lambda x: x.lower())
df_train['comment'] = df_train['comment'].apply(remove_punctuation)

In [16]:
df_test['comment'] = df_test['comment'].apply(lambda x: x.lower())
df_test['comment']= df_test['comment'].apply(remove_punctuation)

In [17]:
df_dev['comment'] = df_dev['comment'].apply(lambda x: x.lower())
df_dev['comment']= df_dev['comment'].apply(remove_punctuation)

In [18]:
df_train['comment'] = df_train['comment'].apply(remove_stopword)
df_test['comment']= df_test['comment'].apply(remove_stopword)
df_dev['comment']= df_dev['comment'].apply(remove_stopword)

In [19]:
df_train['comment'].values[95:98]

array(['điện_thoại khá tốt pin trâu khá mượt bắt wifi cực tốt thỉnh_thoảng máy đơ thoát máy ko cập_nhật miul 12',
       'nghe bảo điện_thoại chơi game nóng mua về chiến liên_quân 3 4h liên_tục thấy ấm nóng lời đồn thứ khác vũng chẳng nói quá ngon',
       '1 máy chơi game lướt web ít sử_dụng giữ pin cao nhất khoảng 1 ngày 16 tiếng tính luôn cả ngủ nhé ngủ chẳng động tới 2 ngày mn nói đâu 2 nhạc nghe cà_giật cà_giật khó_chịu luôn 3 quay video mờ_chán còn okie'],
      dtype=object)

In [20]:
df_train['comment']=df_train['comment'].apply(remove_repeated_words)
df_test['comment']=df_test['comment'].apply(remove_repeated_words)
df_dev['comment']=df_dev['comment'].apply(remove_repeated_words)

In [21]:
df_train['comment'].values[95:98]

array(['điện_thoại khá tốt pin trâu khá mượt bắt wifi cực tốt thỉnh_thoảng máy đơ thoát máy ko cập_nhật miul 12',
       'nghe bảo điện_thoại chơi game nóng mua về chiến liên_quân 3 4h liên_tục thấy ấm nóng lời đồn thứ khác vũng chẳng nói quá ngon',
       '1 máy chơi game lướt web ít sử_dụng giữ pin cao nhất khoảng 1 ngày 16 tiếng tính luôn cả ngủ nhé ngủ chẳng động tới 2 ngày mn nói đâu 2 nhạc nghe cà_giật khó_chịu luôn 3 quay video mờ_chán còn okie'],
      dtype=object)

In [22]:
def viet_tat_to_full(text):
    # Từ điển chứa các từ viết tắt và dạng đầy đủ
    viet_tat_dict = {
        "chs": "chơi",
        "ko": "không",
        "ok": "ổn",
        "k": "không",
        "dc": "được",
        "e": "em",
        "vs": "với",
        "fb": "Facebook",
        "sp": "sản phẩm",
        "dt": "điện thoại",
        "j": "gì",
        "r": "rồi",
        "ss": "so sánh",
        "sd": "sử dụng",
        "nv": "nhân viên",
        "ae": "anh em",
        "mn": "mọi người",
        "hd": "hoạt động",
        "kg": "kilogram",
        "mk": "mình",
        "v": "vậy",
        "h": "giờ",
        "bt": "bình thường",
        "m": "mình",
        "a": "anh",
        "cx": "cũng",
        "kh": "không",
        "lq": "liên quan",
        "ng": "người",
        "ip": "Iphone",
        "bh": "bây giờ",
        "hk": "hong",
        "tr": "triệu",
        "ad": "admin",
        "wa": "qua",
        "ms": "mới",
        "dk": "điều kiện",
        "t": "tôi",
        "vn": "Việt Nam",
        "d": "đi",
        "b": "bạn",
        "ns": "nói",
        "tp": "thành phố",
        "tn": "tin nhắn",
        "s": "sao",
        "se": "sẽ",
        "gg": "Google",
        "of": "của",
        "up": "lên",
        "z": "vậy",
        "la": "là",
        "nt": "nhắn tin",
        "qc": "quảng cáo",
        "on": "online",
        "sx": "sản xuất",
        "vv": "vân vân",
        "y": "ý",
        "c": "cái",
        "lm": "làm",
        "ah": "à",
        "no": "không",
        "nc": "nói chuyện",
        "gb": "goodbye",
        "n": "nhé",
        "cv": "công việc",
        "wf": "wifi",
        "gi": "gì",
        "va": "và",
        "de": "để",
        "zl": "zalo",
        "kb": "không biết",
        "ik": "đi",
        "ji": "gì",
        "co": "có",
        "ak": "à",
        "tg": "thời gian",
        "bi": "bị",
        "bn": "bạn",
        "vi": "vì",
        "o": "ở",
        "tt": "tình trạng",
        "um": "ừm",
        "yt": "YouTube",
        "ps": "postscript",
        "id": "ID",
        "p": "phần",
        "g": "gram",
        "tv": "television",
        "xl": "xin lỗi",
        "cu": "cứ",
        "da": "đã",
        "tl": "trả lời",
        "x": "xem",
        "xg": "xong",
        "xi": "xì",
        "l": "lít",
        "in": "inch",
        "ve": "về",
        "th": "thôi",
        "cg": "cũng",
        "it": "ít",
        "pk": "phải không",
        "bo": "bỏ",
        "bk": "biết",
        "ch": "chưa",
        "ht": "hết",
        "xr": "xong rồi",
        "kq": "kết quả",
        "ib": "nhắn tin",
        "vd": "ví dụ",
        "ao": "áo",
        "ap": "áp",
        "hn": "hôm nay",
        "su": "sự",
        "re": "rẻ",
        "dp": "đẹp",
        "gt": "giới thiệu",
        "vc": "việc",
        "bb": "bạn bè",
        "cn": "Chủ nhật",
        "du": "đủ",
        "tu": "từ",
        "hi": "xin chào",
        "pt": "phát triển",
        "km": "kilomet",
        "by": "bye",
        "or": "hoặc",
        "mb": "mấy bạn",
        "ke": "kể",
        "lp": "lập",
        "bu": "bù",
        "tq": "trung quốc",
        "lg": "lớn",
        "oy": "gòi",
        "vu": "vụ",
        "bl": "bình luận",
        "xo": "xô",
        "ro": "rõ",
        "hs": "học sinh",
        "uu": "ưu",
        "ru": "rủ",
        "lx": "lâu",
        "ag": "An Giang",
        "vo": "vô",
        "sm": "small",
        "vl": "vãi",
        "ol": "online",
        "ky": "ký",
        "pc": "personal computer",
        "oi": "ơi",
        "jo": "giờ",
        "mp": "mp3",
        "nh": "nhà",
        "uk": "ử",
        "pr": "public relations",
        "vk": "vợ",
        "tc": "tình cảm",
        "qr": "QR code",
        "kt": "kết thúc",
        "tb": "thông báo",
        "sf": "so far",
        "gh": "giao hàng",
        "hg": "không",
        "cb": "chuẩn bị",
        "nx": "nữa",
        "qq": "quần què",
        "bd": "bắt đầu",
        "ni": "này"
    }
    
    # Tách văn bản thành các từ riêng lẻ
    words = text.split()
    
    # Thay thế từ viết tắt bằng dạng đầy đủ
    for i in range(len(words)):
        if words[i] in viet_tat_dict:
            words[i] = viet_tat_dict[words[i]]
    
    # Ghép các từ lại thành câu
    return ' '.join(words)


# Áp dụng hàm cho cột 'input' trong DataFrame
df_train["comment"] = df_train["comment"].apply(viet_tat_to_full)

# Hiển thị DataFrame sau khi chuyển đổi
print(df_train["comment"])

0       mới mua máy thegioididong thốt_nốt cảm_thấy ổn...
1       pin kém còn miễn chê mua 8 3 2019 tình_trạng p...
2       sao gọi điện_thoại màn_hình chấm nhỏ nháy gần ...
3       mọi người cập_nhật phần_mềm nó bớt tốn pin mìn...
4       mới mua sài 1 tháng thấy pin trâu sài bao mượt...
                              ...                        
7781    8g đi đánh mạng giật không chịu nổi chán pig c...
7782    mua điều kiện giảm 500k lỗi lòi hết treo màn_h...
7783    máy sài 3 tháng ổn pin trâu khỏi nói sài cả ng...
7784    tiếc hàng realme không ốp lưng ngoài khó xài c...
7785    mình thất_vọng mua máy bắt wifi cực_kì yếu sam...
Name: comment, Length: 7786, dtype: object


In [23]:
def viet_tat_to_full(text):
    # Từ điển chứa các từ viết tắt và dạng đầy đủ
    viet_tat_dict = {
    "đt": "điện thoại",
    "ko": "không",
    "ok": "ổn",
    "k": "không",
    "dc": "được",
    "e": "em",
    "vs": "với",
    "sp": "sản phẩm",
    "fb": "Facebook",
    "dt": "điện thoại",
    "r": "rồi",
    "j": "gì",
    "nv": "nhân viên",
    "ss": "so sánh",
    "mk": "mình",
    "hd": "hoạt động",
    "sd": "sử dụng",
    "mn": "mọi người",
    "ae": "anh em",
    "v": "vậy",
    "lq": "liên quan",
    "h": "giờ",
    "lo": "lo",
    "ip": "Iphone",
    "s": "sao",
    "ms": "mới",
    "ng": "người",
    "ad": "admin",
    "cx": "cũng",
    "bt": "bình thường",
    "kh": "không",
    "a": "anh",
    "kg": "kilogram",
    "dk": "điều kiện",
    "m": "mình",
    "t": "tôi",
    "wf": "wifi",
    "hk": "hong",
    "wa": "qua",
    "la": "là",
    "vn": "Việt Nam",
    "co": "có",
    "ma": "mà",
    "ch": "chưa",
    "ns": "nói",
    "z": "vậy",
    "va": "và",
    "nt": "nhắn tin",
    "d": "dậy",
    "tr": "triệu",
    "gb": "gigabyte",
    "vv": "vân vân",
    "of": "của",
    "ôk": "ổn",
    "đc": "được",
    "mh": "mấy giờ",
    "c": "cái",
    "ak": "à",
    "bh": "bây giờ",
    "x": "xem",
    "gg": "Google",
    "gi": "gì",
    "y": "ý",
    "o": "ở",
    "on": "online",
    "bn": "bạn",
    "in": "inch",
    "up": "nâng cấp",
    "da": "đã",
    "im": "im",
    "id": "ID",
    "lm": "làm",
    "bi": "bị",
    "mi": "mì",
    "tt": "tình trạng",
    "mp": "mp3",
    "tp": "thành phố",
    "pk": "phòng khách",
    "nc": "nói chuyện",
    "ah": "à",
    "ji": "gì",
    "tg": "thời gian",
    "se": "sẽ",
    "b": "bạn",
    "f": "phải",
    "ht": "hoàn thành",
    "ck": "chuyển khoản",
    "g": "giờ",
    "n": "nhé",
    "tu": "tự",
    "qa": "qua",
    "nx": "nữa",
    "sx": "sản xuất",
    "vl": "vãi",
    "de": "để",
    "tq": "trung quốc",
    "cv": "công việc",
    "ro": "rõ",
    "fm": "file manager",
    "bb": "bye bye",
    "kb": "keyboard",
    "vk": "vợ",
    "vc": "việc",
    "iu": "yêu",
    "vi": "vì",
    "vd": "ví dụ",
    "tn": "tin nhắn",
    "or": "hoặc",
    "cs": "có",
    "yt": "YouTube",
    "cu": "cứ",
    "ap": "áp",
    "th": "tháng",
    "zl": "zalo",
    "i": "y",
    "cg": "cũng",
    "km": "kilomet",
    "bk": "bếp",
    "hz": "hertz",
    "ho": "ho",
    "dx": "đấy",
    "jo": "giờ",
    "pb": "phiên bản",
    "kk": "kaka",
    "oi": "ơi",
    "pv": "phỏng vấn",
    "qc": "quảng cáo",
    "os": "operating system",
    "it": "ít",
    "le": "lẽ",
    "ni": "này",
    "w": "with",
    "sn": "sinh nhật",
    "ki": "kì",
    "lg": "lượng",
    "hg": "hông",
    "hi": "xin chào",
    "cn": "Chủ nhật",
    "sv": "sinh viên",
    "bj": "bị",
    "xg": "xong",
    "ln": "luôn",
    "xr": "xin",
    "mb": "mấy bạn",
    "za": "zalo",
    "u": "you",
    "hs": "học sinh",
    "am": "am",
    "vu": "vụ",
    "qr": "QR code",
    "re": "rẻ",
    "pt": "phát triển",
    "bo": "bỏ",
    "ly": "ly",
    "zo": "dô",
    "ny": "nay",
    "su": "sự",
    "kt": "kết thúc",
    "no": "không",
    "bx": "bổ xung",
    "vo": "vô",
    "bc": "báo cáo",
    "tl": "trả lời",
    "ja": "giá",
    "di": "đi",
    "zs": "zs",
    "ls": "lịch sử",
    "qq": "quá",
    "bg": "bây giừo",
    "ac": "anh chị",
    "ky": "ký",
    "q": "quý",
    "ty": "tình yêu",
    "ol": "online",
    "we": "chúng ta",
    "l": "lít",
    "ik": "đi",
    "tk": "tài khoản",
    "tb": "thông báo",
    "pg": "page",
    "ao": "áo",
    "hp": "hạnh phúc",
    "op": "op",
    "ud": "ud",
    "is": "là",
    "pr": "public relations",
    "wp": "wordpress",
    "pl": "làm ơn",
    "cm": "centimeter",
    "rv": "review",
    "na": "na",
    "hc": "học",
    "ha": "ha",
    "hw": "hardware",
    "nh": "nhà",
    "xi": "xì",
    "ti": "ti",
    "ze": "de",
    "hm": "hừm",
    "xm": "xem",
    "db": "database",
    "li": "li",
    "te": "te",
    "fp": "fp",
    "ju": "ju",
    "pj": "pj",
    "kv": "kv",
    "st": "status",
    "ri": "ri",
    "uc": "uc",
    "td": "td",
    "mu": "mu",
    "mo": "mo",
    "ps": "postscript",
    "ke": "kể",
    "he": "hè",
    "hy": "hy",
    "fi": "fi",
    "p": "phần",
    "my": "my",
    "rc": "rc",
    "tx": "tx",
    "dg": "đang",
    "xp": "xp",
    "mx": "mx",
    "ig": "Instagram",
    "uj": "uj",
    "ve": "về",
    "ds": "danh sách",
    "pn": "pn",
    "al": "al",
    "yd": "yd",
    "dm": "damn",
    "zi": "zi",
    "du": "du",
    "kc": "kc",
    "hf": "hf",
    "dp": "đẹp",
    "qt": "qt",
    "ph": "ph",
    "bl": "bình luận",
    "oh": "oh",
    "cb": "chuẩn bị",
    "vt": "vt",
    "dn": "dn",
    "vp": "vp"
    }
    
    # Tách văn bản thành các từ riêng lẻ
    words = text.split()
    
    # Thay thế từ viết tắt bằng dạng đầy đủ
    for i in range(len(words)):
        if words[i] in viet_tat_dict:
            words[i] = viet_tat_dict[words[i]]
    
    # Ghép các từ lại thành câu
    return ' '.join(words)


# Áp dụng hàm cho cột 'input' trong DataFrame
df_test["comment"] = df_test["comment"].apply(viet_tat_to_full)

# Hiển thị DataFrame sau khi chuyển đổi
print(df_test["comment"])

0       điện thoải ổn facelock cực nhanh vân tay ổn mà...
1       mình mới mua vivo91c tải ứng_dụng games nhanh ...
2       xấu đẹp không biết ưng tgdđ phục_vụ tuyệt_vời ...
3       màn_hình hơi lác chơi game nặng mình chơi ở 2 ...
4       nói_chung máy đẹp màn amoled ổn trong tầm giá ...
                              ...                        
2219    mẫu_mã đẹp lung_linh máy chạy cực nhanh mượt h...
2220    ai giống mình máy sài bình_thường hở xem phim ...
2221    sản_phẩm tốt ai chơi game mang 1 em về chơi nh...
2222    vừa_mới mua xong máy đẹp nhân_viên nhiệt_tình ...
2223    mình mua điện thoại cũ xài tốt pin xuống lâu c...
Name: comment, Length: 2224, dtype: object


In [24]:
def viet_tat_to_full(text):
    # Từ điển chứa các từ viết tắt và dạng đầy đủ
    viet_tat_dict = {
    "đt": "điện thoại",
    "ko": "không",
    "ok": "ổn",
    "k": "không",
    "dc": "được",
    "e": "em",
    "vs": "với",
    "sp": "sản phẩm",
    "fb": "Facebook",
    "dt": "điện thoại",
    "r": "rồi",
    "j": "gì",
    "nv": "nhân viên",
    "ss": "so sánh",
    "mk": "mình",
    "hd": "hoạt động",
    "sd": "sử dụng",
    "mn": "mọi người",
    "ae": "anh em",
    "v": "vậy",
    "lq": "liên quan",
    "h": "giờ",
    "lo": "lo",
    "ip": "Iphone",
    "s": "sao",
    "ms": "mới",
    "ng": "người",
    "ad": "admin",
    "cx": "cũng",
    "bt": "bình thường",
    "kh": "không",
    "a": "anh",
    "kg": "kilogram",
    "dk": "điều kiện",
    "m": "mình",
    "t": "tôi",
    "wf": "wifi",
    "hk": "hong",
    "wa": "qua",
    "la": "là",
    "vn": "Việt Nam",
    "co": "có",
    "ma": "mà",
    "ch": "chưa",
    "ns": "nói",
    "z": "vậy",
    "va": "và",
    "nt": "nhắn tin",
    "d": "dậy",
    "tr": "triệu",
    "gb": "gigabyte",
    "vv": "vân vân",
    "of": "của",
    "ôk": "ổn",
    "đc": "được",
    "mh": "mấy giờ",
    "c": "cái",
    "ak": "à",
    "bh": "bây giờ",
    "x": "xem",
    "gg": "Google",
    "gi": "gì",
    "y": "ý",
    "o": "ở",
    "on": "online",
    "bn": "bạn",
    "in": "inch",
    "up": "nâng cấp",
    "da": "đã",
    "im": "im",
    "id": "ID",
    "lm": "làm",
    "bi": "bị",
    "mi": "mì",
    "tt": "tình trạng",
    "mp": "mp3",
    "tp": "thành phố",
    "pk": "phòng khách",
    "nc": "nói chuyện",
    "ah": "à",
    "ji": "gì",
    "tg": "thời gian",
    "se": "sẽ",
    "b": "bạn",
    "f": "phải",
    "ht": "hoàn thành",
    "ck": "chuyển khoản",
    "g": "giờ",
    "n": "nhé",
    "tu": "tự",
    "qa": "qua",
    "nx": "nữa",
    "sx": "sản xuất",
    "vl": "vãi",
    "de": "để",
    "tq": "trung quốc",
    "cv": "công việc",
    "ro": "rõ",
    "fm": "file manager",
    "bb": "bye bye",
    "kb": "keyboard",
    "vk": "vợ",
    "vc": "việc",
    "iu": "yêu",
    "vi": "vì",
    "vd": "ví dụ",
    "tn": "tin nhắn",
    "or": "hoặc",
    "cs": "có",
    "yt": "YouTube",
    "cu": "cứ",
    "ap": "áp",
    "th": "tháng",
    "zl": "zalo",
    "i": "y",
    "cg": "cũng",
    "km": "kilomet",
    "bk": "bếp",
    "hz": "hertz",
    "ho": "ho",
    "dx": "đấy",
    "jo": "giờ",
    "pb": "phiên bản",
    "kk": "kaka",
    "oi": "ơi",
    "pv": "phỏng vấn",
    "qc": "quảng cáo",
    "os": "operating system",
    "it": "ít",
    "le": "lẽ",
    "ni": "này",
    "w": "with",
    "sn": "sinh nhật",
    "ki": "kì",
    "lg": "lượng",
    "hg": "hông",
    "hi": "xin chào",
    "cn": "Chủ nhật",
    "sv": "sinh viên",
    "bj": "bị",
    "xg": "xong",
    "ln": "luôn",
    "xr": "xin",
    "mb": "mấy bạn",
    "za": "zalo",
    "u": "you",
    "hs": "học sinh",
    "am": "am",
    "vu": "vụ",
    "qr": "QR code",
    "re": "rẻ",
    "pt": "phát triển",
    "bo": "bỏ",
    "ly": "ly",
    "zo": "dô",
    "ny": "nay",
    "su": "sự",
    "kt": "kết thúc",
    "no": "không",
    "bx": "bổ xung",
    "vo": "vô",
    "bc": "báo cáo",
    "tl": "trả lời",
    "ja": "giá",
    "di": "đi",
    "zs": "zs",
    "ls": "lịch sử",
    "qq": "quá",
    "bg": "bây giừo",
    "ac": "anh chị",
    "ky": "ký",
    "q": "quý",
    "ty": "tình yêu",
    "ol": "online",
    "we": "chúng ta",
    "l": "lít",
    "ik": "đi",
    "tk": "tài khoản",
    "tb": "thông báo",
    "pg": "page",
    "ao": "áo",
    "hp": "hạnh phúc",
    "op": "op",
    "ud": "ud",
    "is": "là",
    "pr": "public relations",
    "wp": "wordpress",
    "pl": "làm ơn",
    "cm": "centimeter",
    "rv": "review",
    "na": "na",
    "hc": "học",
    "ha": "ha",
    "hw": "hardware",
    "nh": "nhà",
    "xi": "xì",
    "ti": "ti",
    "ze": "de",
    "hm": "hừm",
    "xm": "xem",
    "db": "database",
    "li": "li",
    "te": "te",
    "fp": "fp",
    "ju": "ju",
    "pj": "pj",
    "kv": "kv",
    "st": "status",
    "ri": "ri",
    "uc": "uc",
    "td": "td",
    "mu": "mu",
    "mo": "mo",
    "ps": "postscript",
    "ke": "kể",
    "he": "hè",
    "hy": "hy",
    "fi": "fi",
    "p": "phần",
    "my": "my",
    "rc": "rc",
    "tx": "tx",
    "dg": "đang",
    "xp": "xp",
    "mx": "mx",
    "ig": "Instagram",
    "uj": "uj",
    "ve": "về",
    "ds": "danh sách",
    "pn": "pn",
    "al": "al",
    "yd": "yd",
    "dm": "damn",
    "zi": "zi",
    "du": "du",
    "kc": "kc",
    "hf": "hf",
    "dp": "đẹp",
    "qt": "qt",
    "ph": "ph",
    "bl": "bình luận",
    "oh": "oh",
    "cb": "chuẩn bị",
    "vt": "vt",
    "dn": "dn",
    "vp": "vp"
    }
    
    # Tách văn bản thành các từ riêng lẻ
    words = text.split()
    
    # Thay thế từ viết tắt bằng dạng đầy đủ
    for i in range(len(words)):
        if words[i] in viet_tat_dict:
            words[i] = viet_tat_dict[words[i]]
    
    # Ghép các từ lại thành câu
    return ' '.join(words)


# Áp dụng hàm cho cột 'input' trong DataFrame
df_dev["comment"] = df_dev["comment"].apply(viet_tat_to_full)

# Hiển thị DataFrame sau khi chuyển đổi
print(df_dev["comment"])

0       máy ổn không chê điểm nào chính_thức fans vivo...
1       mới sài 3 ngày sản_phẩm khá ổn lỗi thấy nghe g...
2       dùng tốt chơi game bao mượt nóng lời đồn pin t...
3       ai giống mấy giờ kilogram mấy giờ mua Iphone 7...
4       không bjk ngày_mai đổi được không mới mua hqua...
                              ...                        
1107    mua về xài hơn tháng giờ sạc pin gửi bảo_hành ...
1108    mới mua samsung a10s sài đầy 1 tháng cảm_ứng n...
1109    máy sử_dụng mượt thiết_kế ổn pin khá tuyệt_vời...
1110    máy mẫu_mã đẹp pin sử_dụng 2 ngày giá phù_hợp ...
1111    mới mua máy biết sử_dụng sao nhân_viên bán hàn...
Name: comment, Length: 1112, dtype: object


In [25]:
df_train.head()

,index,comment,n_star,date_time,label
0,0,mới mua máy thegioididong thốt_nốt cảm_thấy ổn...,5,2 tuần trước,{CAMERA#Positive};{FEATURES#Positive};{BATTERY...
1,1,pin kém còn miễn chê mua 8 3 2019 tình_trạng p...,5,14/09/2019,{BATTERY#Negative};{GENERAL#Positive};{OTHERS};
2,2,sao gọi điện_thoại màn_hình chấm nhỏ nháy gần ...,3,17/08/2020,{FEATURES#Negative};
3,3,mọi người cập_nhật phần_mềm nó bớt tốn pin mìn...,3,29/02/2020,{FEATURES#Negative};{BATTERY#Neutral};{GENERAL...
4,4,mới mua sài 1 tháng thấy pin trâu sài bao mượt...,5,4/6/2020,{BATTERY#Positive};{PERFORMANCE#Positive};{SER...


In [26]:
def count_features(label):
    features = re.findall(r'\{.*?#', label)
    num_features = len(features)
    return num_features

In [27]:
count_features = df_train['label'].apply(count_features)

In [28]:
# Tìm chỉ số của giá trị lớn nhất trong count_features
index_max_features=count_features.idxmax()
number_of_features=df_train.loc[index_max_features,'label']
print(f"Number of features included: {number_of_features}")
print(f"Max features:  {count_features.max()}")

Number of features included: {SCREEN#Positive};{CAMERA#Positive};{FEATURES#Positive};{BATTERY#Positive};{PERFORMANCE#Positive};{DESIGN#Positive};{PRICE#Neutral};{GENERAL#Positive};{SER&ACC#Positive};
Max features:  9


In [29]:
def check_value_features(data):
    for feature in number_of_features:
          if feature in data:
            return True
    return False

In [30]:
df_train['contains_features'] =df_train['label'].apply(lambda x:check_value_features(x))
print(df_train['contains_features'].count())
# Hiển thị các dòng không chứa ít nhất một trong các tính năng được liệt kê
rows_without_features = df_train[~df_train['contains_features']]
print(rows_without_features)

7786
Empty DataFrame
Columns: [index, comment, n_star, date_time, label, contains_features]
Index: []


In [31]:
def count_positive_labels(label):
    return label.count("Positive")
def count_neural_labels(label):
    return label.count("Neutral")
def count_negative_labels(label):
    return label.count("Negative")

In [32]:
df_train['positive_count']=df_train['label'].apply(count_positive_labels)
df_train['neutral_count']=df_train['label'].apply(count_neural_labels)
df_train['negative_count']=df_train['label'].apply(count_negative_labels)
df_test['positive_count']=df_test['label'].apply(count_positive_labels)
df_test['neutral_count']=df_test['label'].apply(count_neural_labels)
df_test['negative_count']=df_test['label'].apply(count_negative_labels)
df_dev['positive_count']=df_dev['label'].apply(count_positive_labels)
df_dev['neutral_count']=df_dev['label'].apply(count_neural_labels)
df_dev['negative_count']=df_dev['label'].apply(count_negative_labels)

In [33]:
def assign_label(row):
    if row['positive_count'] > row['neutral_count'] and row['positive_count'] > row['negative_count']:
        return 'Positive'
    elif row['negative_count'] >row['neutral_count'] and row['negative_count'] > row['positive_count']:
        return 'Negative'
    elif row['negative_count'] == row['neutral_count'] :
        return 'Negative'
    elif row['neutral_count']== row ['positive_count']:
        return "Positive"
    else :
        return "Neutral"

In [34]:
df_train['label'] = df_train.apply(assign_label, axis=1)
df_test['label'] = df_test.apply(assign_label,axis=1)
df_dev['label'] = df_dev.apply(assign_label,axis=1)

In [35]:
df_train.head()

,index,comment,n_star,date_time,label,contains_features,positive_count,neutral_count,negative_count
0,0,mới mua máy thegioididong thốt_nốt cảm_thấy ổn...,5,2 tuần trước,Positive,True,6,0,0
1,1,pin kém còn miễn chê mua 8 3 2019 tình_trạng p...,5,14/09/2019,Neutral,True,1,0,1
2,2,sao gọi điện_thoại màn_hình chấm nhỏ nháy gần ...,3,17/08/2020,Negative,True,0,0,1
3,3,mọi người cập_nhật phần_mềm nó bớt tốn pin mìn...,3,29/02/2020,Neutral,True,0,2,1
4,4,mới mua sài 1 tháng thấy pin trâu sài bao mượt...,5,4/6/2020,Positive,True,2,0,1


In [36]:
df_train.columns

Index(['index', 'comment', 'n_star', 'date_time', 'label', 'contains_features',
       'positive_count', 'neutral_count', 'negative_count'],
      dtype='object')

In [37]:
df_train.shape

(7786, 9)

In [38]:
df_train.head()

,index,comment,n_star,date_time,label,contains_features,positive_count,neutral_count,negative_count
0,0,mới mua máy thegioididong thốt_nốt cảm_thấy ổn...,5,2 tuần trước,Positive,True,6,0,0
1,1,pin kém còn miễn chê mua 8 3 2019 tình_trạng p...,5,14/09/2019,Neutral,True,1,0,1
2,2,sao gọi điện_thoại màn_hình chấm nhỏ nháy gần ...,3,17/08/2020,Negative,True,0,0,1
3,3,mọi người cập_nhật phần_mềm nó bớt tốn pin mìn...,3,29/02/2020,Neutral,True,0,2,1
4,4,mới mua sài 1 tháng thấy pin trâu sài bao mượt...,5,4/6/2020,Positive,True,2,0,1


In [39]:
def is_valid_date(date_str):
    # Biểu thức chính quy để kiểm tra định dạng ngày tháng
    date_pattern = r'\d{1,2}/\d{1,2}/\d{4}'
    return bool(re.match(date_pattern, date_str))

In [40]:
valid_dates = df_train[df_train['date_time'].apply(is_valid_date)]
valid_dates_test = df_test[df_test['date_time'].apply(is_valid_date)]
valid_dates_dev = df_dev[df_test['date_time'].apply(is_valid_date)]

C:\Users\HP\AppData\Local\Temp\ipykernel_2076\2397564853.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valid_dates_dev = df_dev[df_test['date_time'].apply(is_valid_date)]


In [41]:
valid_dates.shape

(6930, 9)

In [42]:
valid_dates.head()

,index,comment,n_star,date_time,label,contains_features,positive_count,neutral_count,negative_count
1,1,pin kém còn miễn chê mua 8 3 2019 tình_trạng p...,5,14/09/2019,Neutral,True,1,0,1
2,2,sao gọi điện_thoại màn_hình chấm nhỏ nháy gần ...,3,17/08/2020,Negative,True,0,0,1
3,3,mọi người cập_nhật phần_mềm nó bớt tốn pin mìn...,3,29/02/2020,Neutral,True,0,2,1
4,4,mới mua sài 1 tháng thấy pin trâu sài bao mượt...,5,4/6/2020,Positive,True,2,0,1
5,5,xài tốt mượt pin trâu bạn độ sáng đủ nhân_viên...,5,20/06/2019,Positive,True,3,2,0


In [43]:
invalid_dates=df_train[~df_train['date_time'].apply(is_valid_date)]
invalid_dates_test=df_test[~df_test['date_time'].apply(is_valid_date)]
invalid_dates_dev=df_dev[~df_dev['date_time'].apply(is_valid_date)]

In [44]:
invalid_dates.shape

(856, 9)

In [45]:
invalid_dates.head(5)

,index,comment,n_star,date_time,label,contains_features,positive_count,neutral_count,negative_count
0,0,mới mua máy thegioididong thốt_nốt cảm_thấy ổn...,5,2 tuần trước,Positive,True,6,0,0
6,6,mình mới xài 7 tháng xuống 7 pin chả hiểu máy ...,1,1 tuần trước,Negative,True,0,0,1
11,11,khá ổn trong tầm giá cam đẹp sạc nhanh màn_hìn...,5,3 tuần trước,Positive,True,4,2,0
14,14,mình mua được 1 tuần máy phát trực_tiếp tự_nhi...,1,6 ngày trước,Negative,True,0,0,3
18,18,dùng đơ máy bấm mãi mới khởi_động song nóng ra...,1,5 ngày trước,Negative,True,0,0,1


In [46]:
common_value=valid_dates['date_time'].mode()[0]
common_value_test=valid_dates_test['date_time'].mode()[0]
common_value_dev=valid_dates_dev['date_time'].mode()[0]

invalid_dates.loc[:, 'date_time'] = common_value
invalid_dates_test.loc[:,'date_time']=common_value
invalid_dates_dev.loc[:,'date_time']=common_value

In [47]:
invalid_dates.head()

,index,comment,n_star,date_time,label,contains_features,positive_count,neutral_count,negative_count
0,0,mới mua máy thegioididong thốt_nốt cảm_thấy ổn...,5,21/04/2020,Positive,True,6,0,0
6,6,mình mới xài 7 tháng xuống 7 pin chả hiểu máy ...,1,21/04/2020,Negative,True,0,0,1
11,11,khá ổn trong tầm giá cam đẹp sạc nhanh màn_hìn...,5,21/04/2020,Positive,True,4,2,0
14,14,mình mua được 1 tuần máy phát trực_tiếp tự_nhi...,1,21/04/2020,Negative,True,0,0,3
18,18,dùng đơ máy bấm mãi mới khởi_động song nóng ra...,1,21/04/2020,Negative,True,0,0,1


In [48]:
df_train=pd.concat([valid_dates,invalid_dates],ignore_index=True)
df_test=pd.concat ([valid_dates_test,invalid_dates_test],ignore_index=True)
df_dev=pd.concat ([valid_dates_dev,invalid_dates_dev],ignore_index=True)

In [49]:
df_dev.shape

(1111, 8)

In [50]:
df_dev.head()

,index,comment,n_star,date_time,label,positive_count,neutral_count,negative_count
0,0,máy ổn không chê điểm nào chính_thức fans vivo...,5,9/7/2020,Positive,2,0,0
1,1,mới sài 3 ngày sản_phẩm khá ổn lỗi thấy nghe g...,5,4/5/2020,Positive,3,1,1
2,2,dùng tốt chơi game bao mượt nóng lời đồn pin t...,5,14/12/2019,Positive,4,0,0
3,3,ai giống mấy giờ kilogram mấy giờ mua Iphone 7...,3,7/3/2019,Negative,0,0,1
4,4,không bjk ngày_mai đổi được không mới mua hqua...,2,29/03/2020,Negative,0,0,1


In [51]:
df_train.drop(["index",'contains_features'],axis=1,inplace=True)
df_test.drop(['index'],axis=1,inplace=True)
df_dev.drop(['index'],axis=1,inplace=True)

In [52]:
df_train.shape

(7786, 7)

In [53]:
df_train.head()

,comment,n_star,date_time,label,positive_count,neutral_count,negative_count
0,pin kém còn miễn chê mua 8 3 2019 tình_trạng p...,5,14/09/2019,Neutral,1,0,1
1,sao gọi điện_thoại màn_hình chấm nhỏ nháy gần ...,3,17/08/2020,Negative,0,0,1
2,mọi người cập_nhật phần_mềm nó bớt tốn pin mìn...,3,29/02/2020,Neutral,0,2,1
3,mới mua sài 1 tháng thấy pin trâu sài bao mượt...,5,4/6/2020,Positive,2,0,1
4,xài tốt mượt pin trâu bạn độ sáng đủ nhân_viên...,5,20/06/2019,Positive,3,2,0


In [54]:
df_train['comment'][95]

'mới_đầu sử dụng thi ổn cảm_ứng chạm giật lác treo máy thường_xuyên gọi zalo ma treo máy nghĩ chán'

In [55]:
df_train.to_csv("G:\\VietNamese-Sentiment-Analyst\\data\\data_stopword\\train_process.csv", index=False)
df_test.to_csv("G:\\VietNamese-Sentiment-Analyst\\data\\data_stopword\\test_process.csv",index=False)
df_dev.to_csv("G:\\VietNamese-Sentiment-Analyst\\data\\data_stopword\\dev_process.csv",index=False)